## 지니차트 Top100

In [76]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

- Step 1.
    - Url 가져오기

In [77]:
url = 'https://www.genie.co.kr/chart/top200'
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
soup = BeautifulSoup(res.text, 'html.parser')


In [78]:
soup.select_one('#inc_date')            # 날짜 가져오기

<span id="inc_date">2013.03.06</span>

In [79]:
from selenium import webdriver      # 크롬을 사용하여 불러오면 현재 지니에 적용된 시간을 불러올 수 있음

driver = webdriver.Chrome() 
driver.get(url)

In [80]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
date = soup.select_one('#inc_date')
date


<span id="inc_date">2023.09.19</span>

In [81]:

time = soup.select_one('#inc_time')
time

<span id="inc_time">10:00</span>

- Step2
    - 원하는 데이터 찾기 

In [82]:
trs = soup.select('tr.list')        
len(trs)            # 한 페이지에 50개 곡

50

- Step3 
    - 하나를 선택해서 원하는 정보 추출

In [83]:
tr = trs[0]
# '1\n                                        \n
rank = tr.select_one('.number').get_text().split('\n')[0].strip()   # '1'만 필요하기 때문에 스플릿, 스트립
rank

'1'

In [84]:
title = tr.select_one('.albumtitle.ellipsis').get_text().strip()
title

'Love Lee'

In [85]:
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'AKMU (악뮤)'

In [86]:
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'Love Lee'

In [87]:
img = tr.select_one('tr > td:nth-child(3) > a > img')['src']
img

'//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/610/84181610_1692585955705_1_140x140.JPG/dims/resize/Q_80,0'

- Step4 

In [88]:
data = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()
    title = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    img = tr.select_one('tr > td:nth-child(3) > a > img')['src']
    data.append({'순위':rank, '제목':title, '아티스트':artist, '앨범':album, '이미지':img})

- Step 5: 생략

- Step 6 
    - 1페이지부터 5페이지까지 출력 ( 반복문 사용 )

In [89]:
data = []
for page in tqdm(range(1,5)):
    header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh={time}&rtm=Y&pg={page}'
    res = requests.get(url, headers=header)
    soup = BeautifulSoup(res.text, 'html.parser')
    trs = soup.select('tr.list') 

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()
        title = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        img = tr.select_one('tr > td:nth-child(3) > a > img')['src']
        data.append({'순위':rank, '제목':title, '아티스트':artist, '앨범':album, '이미지':img})



100%|██████████| 4/4 [00:02<00:00,  1.47it/s]


- Step 7

In [90]:
df = pd.DataFrame(data)
df

,순위,제목,아티스트,앨범,이미지
0,1,Love Lee,AKMU (악뮤),Love Lee,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/...
1,2,스트릿 우먼 파이터2 (SWF2) 계급미션,다이나믹 듀오 & 이영지,스트릿 우먼 파이터2 (SWF2) 계급미션,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/193/...
2,3,Love Lee,AKMU (악뮤),Love Lee,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/...
3,4,NewJeans 2nd EP 'Get Up',NewJeans,NewJeans 2nd EP 'Get Up',//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/019/...
4,5,Seven (Clean Ver.),정국,Seven (Clean Ver.),//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/048/...
...,...,...,...,...,...
195,196,In The Lonely Hour (Standard Edition),Sam Smith,In The Lonely Hour (Standard Edition),//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/080/437/...
196,197,혼자라고 생각말기,임한별,혼자라고 생각말기,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/900/...
197,198,Take,숀 (SHAUN),Take,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/081/078/...
198,199,Get A Guitar,RIIZE,Get A Guitar,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/...


In [91]:
date = date.string.replace('.','')
time = time.string.replace(':','')
filename = date + time + '.csv'
filename

'202309191000.csv'

In [94]:
df.to_csv('data/지니차트TOP200_'+filename, index=False)
pd.read_csv('data/지니차트TOP200_'+filename).tail()

,순위,제목,아티스트,앨범,이미지
195,196,In The Lonely Hour (Standard Edition),Sam Smith,In The Lonely Hour (Standard Edition),//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/080/437/...
196,197,혼자라고 생각말기,임한별,혼자라고 생각말기,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/900/...
197,198,Take,숀 (SHAUN),Take,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/081/078/...
198,199,Get A Guitar,RIIZE,Get A Guitar,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/...
199,200,사랑이라 쓰고 이별이라 읽어,송하예,사랑이라 쓰고 이별이라 읽어,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/245/...
